In [6]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

In [7]:
def redo_cs_names(df):
    df["Case Study"] = np.where(df["Case Study"] == "pulp_and_paper", "Pulp & Paper", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "iron_and_steel", "Steel Plant", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "upw", "Fab25", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "ashkelon", "Ashkelon", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "carlsbad", "Carlsbad", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "santa_barbara", "Santa Barbara", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "tampa_bay", "Tampa Bay", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "big_spring", "Big Spring", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "hrsd", "Hampton Roads", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "ocwd", "Orange County", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "solaire", "Solaire", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "emwd", "EMWD", df["Case Study"]) 
    df["Case Study"] = np.where(df["Case Study"] == "irwin", "Irwin", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "kbhdp", "KBHDP", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "heap_leaching", "Heap Leaching", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "lithium", "Lithium", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "uranium", "Uranium", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "cherokee", "Cherokee", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "gila_river", "Gila River", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "beef", "Beef Processing", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "monterey_one", "Monterey One", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "san_luis", "San Luis TD", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "kern_county", "SJC Kern County", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "damodar", "Damodar", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "produced_water_injection", "SWD Permian Basin", df["Case Study"])
    
    return df

In [8]:
def remove_unnamed(df):
    if "Unnamed: 0" in df.columns:
        del df["Unnamed: 0"]
    if "Unnamed: 0.1" in df.columns:
        del df["Unnamed: 0.1"]
    return df

In [9]:
def set_nan(df):

    df.ByInletCapacity = np.where(df.Variable == "System Levelized Cost", np.nan, df.ByInletCapacity)
    df.ByInletCapacity = np.where(df.Variable == "Water Recovery", np.nan, df.ByInletCapacity)
    df.ByInletCapacity = np.where(df.Variable == "Electricity Intensity", np.nan, df.ByInletCapacity)
    df.ByTreatedCapacity = np.where(df.Variable == "Water Recovery", np.nan, df.ByTreatedCapacity)
    df.ByTreatedCapacity = np.where(df.Variable == "System Levelized Cost", np.nan, df.ByTreatedCapacity)
    df.ByTreatedCapacity = np.where(df.Variable == "Electricity Intensity", np.nan, df.ByTreatedCapacity)
    
    return df

In [10]:
#source_water = "brackish"
df1 = pd.read_csv("data/baseline_cases_runs.csv")
for source_water in ["brackish"]: #df1.source_water_category.unique():

    print(source_water)
    df = pd.read_csv("data/baseline_cases_runs.csv")

    os.makedirs("results/case_studies/%s" % source_water)
    os.makedirs("results/case_studies/%s/baseline" % source_water)
    os.makedirs("results/case_studies/%s/baseline_and_whatifs" % source_water)
    os.makedirs("results/case_studies/%s/sensitivities" % source_water)
    os.makedirs("results/case_studies/%s/baseline/figures" % source_water)
    os.makedirs("results/case_studies/%s/baseline_and_whatifs/figures" % source_water)
    os.makedirs("results/case_studies/%s/baseline/csvs" % source_water)
    os.makedirs("results/case_studies/%s/baseline_and_whatifs/csvs" % source_water)
    os.makedirs("results/case_studies/%s/baseline_and_whatifs/by_unit" % source_water)
    os.makedirs("results/case_studies/%s/baseline/by_unit" % source_water)
    
    cases = df[df.source_water_category == source_water].case_study.unique()
    
    for case in cases:
        case_wi_df = pd.DataFrame()
        path = './results/case_studies/'
        files = []
        for i in os.listdir(path):
            if os.path.isfile(os.path.join(path,i)) and case in i:
                if 'baseline' in i:
                    continue
                if "whatifs" in i:
                    continue
                if "sensitivity" in i:
                    continue
                else:
                    print(case, i)
                    files.append(i)

        for file in files:
            df = pd.read_csv('%s%s' % (path, file), index_col=False)
            case_wi_df = pd.concat([case_wi_df,df])

        #del case_wi_df["Unnamed: 0"]
        remove_unnamed(case_wi_df)
        #case_wi_df.to_csv("results/case_studies/%s/%s_whatifs.csv" % (source_water, case), index=False)

        # combining whatifs and baseline
        baseline_df = pd.read_csv('results/case_studies/%s_baseline.csv' % case, index_col=False)
        remove_unnamed(baseline_df)
        baseline_df.to_csv("results/case_studies/%s/baseline/%s_baseline.csv" % (source_water, case), index=False)
        baseline_df = pd.concat([baseline_df,case_wi_df])

        remove_unnamed(baseline_df)
        #del baseline_df["Unnamed: 0"]
        baseline_df.to_csv("results/case_studies/%s/baseline_and_whatifs/%s_baseline_and_whatifs.csv" % (source_water, 
                                                                                                         case), index=False)
        
    print(source_water, cases)
        
    #save_figs(cases, source_water)
    
#def save_figs(cases, source_water):
    
    #print(source_water)
    #print(cases)

    for file_type in ["baseline", "baseline_and_whatifs"]: #whatifs
        # combine baseline results and clean
        df = pd.DataFrame()

        if len(cases) == 2:
            df1 = pd.read_csv(("results/case_studies/%s/%s/%s_%s.csv" % (source_water, file_type, cases[0], file_type)), index_col=False)
            df2 = pd.read_csv(("results/case_studies/%s/%s/%s_%s.csv" % (source_water, file_type, cases[1], file_type)), index_col=False)
            df = pd.concat([df1, df2])
        if len(cases) == 3:
            df1 = pd.read_csv(("results/case_studies/%s/%s/%s_%s.csv" % (source_water, file_type, cases[0], file_type)), index_col=False)
            df2 = pd.read_csv(("results/case_studies/%s/%s/%s_%s.csv" % (source_water, file_type, cases[1], file_type)), index_col=False)
            df3 = pd.read_csv(("results/case_studies/%s/%s/%s_%s.csv" % (source_water, file_type, cases[2], file_type)), index_col=False)
            df = pd.concat([df1, df2, df3])
        if len(cases) == 4:
            df1 = pd.read_csv(("results/case_studies/%s/%s/%s_%s.csv" % (source_water, file_type, cases[0], file_type)), index_col=False)
            df2 = pd.read_csv(("results/case_studies/%s/%s/%s_%s.csv" % (source_water, file_type, cases[1], file_type)), index_col=False)
            df3 = pd.read_csv(("results/case_studies/%s/%s/%s_%s.csv" % (source_water, file_type, cases[2], file_type)), index_col=False)
            df4 = pd.read_csv(("results/case_studies/%s/%s/%s_%s.csv" % (source_water, file_type, cases[3], file_type)), index_col=False)
            df = pd.concat([df1, df2, df3, df4])
        
        print(df.head())
        
        if source_water == "conventional":
            df_names = pd.read_csv("data/conv_names.csv")
        else:
            df_names = pd.read_excel("data/Category_lookup.xlsx", sheet_name='Sheet1')
        
        
        df_names = df_names[df_names.CaseStudy.isin(cases)]
        df_names = df_names.set_index(df_names.caseunit)
        df["case_name"] = df["Case Study"] + df["Scenario"] + df["Unit Process Name"] # WILL NEED TO EDIT FOR OTHER WHATIFS
        df["Treatment Category"] = df.case_name.map(df_names.category)
        remove_unnamed(df)
        
        df = df[df["Case Study"] != "solaire"]
        
        df = redo_cs_names(df)
        
        # delete unwanted variables
        d_vars = ['Land', 'Working Capital', 'Fixed Capital Investment (FCI)', 
                 'Employee Salaries', 'Benefits', 'Maintenance', 'Laboratory', 'Insurance and Taxes', 
                 'Base Employee Salary', 'System Total Operating', 'System Fixed Operating',
                 'System Other Variable Operating', 'System Total Operating', "Electricity Fraction of LCOW"]

        for d_var in d_vars:
            df = df[df.Variable != d_var]

        df = df.drop_duplicates()

        df.Variable = np.where(df.Variable == 'Other Operating', "Other Variable Operating", df.Variable)
        df.Variable = np.where(df.Variable == 'Total Fixed Operating ', 'Fixed Operating', df.Variable)
        remove_unnamed(df)
        df.to_csv("results/case_studies/%s/%s/%s_%s_results.csv" % (source_water, file_type, source_water,file_type))
        remove_unnamed(df)

        df["CaseStudy_Scenario"] = np.where(df.Scenario == "baseline", df["Case Study"],
                                            df["Case Study"] + " " + df.Scenario)

        
        ###########################################################################
        ###########################################################################
        
        df_unit = pd.read_excel("data/treatment_train_setup.xlsx", sheet_name='units')
        df_unit["id"] = df_unit.CaseStudy + df_unit.Scenario + df_unit.UnitName
        df_unit = df_unit.set_index(df_unit.id)
        df_unit = df_unit[df_unit['id'].notna()]
        df["type"] = df.case_name.map(df_unit.Type)

        df["ByInletCapacity"] = np.array(0)
        df["ByTreatedCapacity"]  = np.array(0)

        for cs in df.CaseStudy_Scenario.unique():
            print("adding inlet/outlet values", cs)

            inlet = df[((df.CaseStudy_Scenario == cs) & (df.Variable == "Inlet Water") & (df.type == "intake"))].Value.sum()
            outlet = df[((df.CaseStudy_Scenario == cs) & (df.Variable == "Inlet Water") & (df.type == "use"))].Value.sum()
            df["ByInletCapacity"] = np.where(df.CaseStudy_Scenario == cs, df.Value / inlet, df.ByInletCapacity)
            df["ByTreatedCapacity"] = np.where(df.CaseStudy_Scenario == cs, df.Value / outlet, df.ByTreatedCapacity)
        
        df = set_nan(df)
        
        ###########################################################################
        ###########################################################################

        
        ### Graphs

        sys_df = df[df["Unit Process Name"] == "System"]
        sys_df = sys_df[sys_df["Metric"] == "LCOW"]
        sys_df = sys_df[sys_df["Variable"] != "System Levelized Cost"]
        sys_df = sys_df[sys_df["Variable"] != "Electricity Fraction of LCOW"]
        sys_df = sys_df.sort_values("CaseStudy_Scenario", ascending=True)

        fig = px.bar(sys_df, x="CaseStudy_Scenario", y="Value", color="Variable",
                      labels={"Variable": "Portion of LCOW",  "Value": "Levelized Cost of Water ($/m3)"},
                      color_discrete_sequence=px.colors.qualitative.T10,
                    template="simple_white")
        
        fig.update_layout(font=dict(size=18))
        
        fig.show()
        fig.write_image("results/case_studies/%s/%s/figures/LCOW_Cost_Category.png" % (source_water,file_type),
                       format="png", engine="kaleido",
                       width=900, height=700, scale=1.7)
        
        sys_df.to_csv("results/case_studies/%s/%s/csvs/LCOW_Cost_Category.csv" % (source_water,file_type))
        
        for metric in ["Unit Levelized Cost", "Total Capital Investment (TCI)", "Annual O&M Costs", 
                      "Electricity Intensity System Treated"]:

            if metric == "Unit Levelized Cost": ylabel = "Levelized Cost of Water ($/m3)"
            if metric == "Total Capital Investment (TCI)": ylabel = "Total Capital Investment ($MM)"
            if metric == "Annual O&M Costs": ylabel = "Annual O&M Costs ($MM/yr)"
            if metric == "Electricity Intensity System Treated": ylabel = "Electricity Intensity kwh/m3 of Treated Water"

            sys_df = df[df["Unit Process Name"] != "System"]
            sys_df = sys_df[sys_df.Variable == metric]
            sys_df = sys_df.groupby(["CaseStudy_Scenario", "Treatment Category"]).sum()
            sys_df = sys_df.reset_index()
            sys_df = sys_df.sort_values("CaseStudy_Scenario", ascending=True)

            fig = px.bar(sys_df, x="CaseStudy_Scenario", y="Value", color="Treatment Category",
                          labels={"Variable": "",  "Value": ylabel},
                         color_discrete_sequence=px.colors.qualitative.T10,
                        template="simple_white")

            fig.update_layout(font=dict(size=18))
            
            #fig.add_vrect(x0=0.9, x1=2)
            if metric == "Unit Levelized Cost": metric = "Treatment Category LCOW"

            fig.write_image("results/case_studies/%s/%s/figures/%s_%s.png" % 
                            (source_water,file_type, metric, file_type),
                                              format="png", engine="kaleido",
                       width=900, height=700, scale=1.7)
            sys_df.to_csv("results/case_studies/%s/%s/csvs/%s_%s.csv" % 
                          (source_water, file_type, metric, file_type))
            fig.show()
            
            if metric in ["Total Capital Investment (TCI)", "Annual O&M Costs"]:
                
                ylabel1 = ylabel + "per_m3_inlet"
                
                fig = px.bar(sys_df, x="CaseStudy_Scenario", y="ByInletCapacity", color="Treatment Category",
                              labels={"Variable": "",  "ByInletCapacity": ylabel1},
                             color_discrete_sequence=px.colors.qualitative.T10,
                            template="simple_white")
                
                fig.update_layout(font=dict(size=18))
                
                fig.write_image("results/case_studies/%s/%s/figures/%s.png" % 
                                (source_water,file_type,(metric+"per_m3_inlet")), 
                                format="png", engine="kaleido", width=900, height=700, scale=1.7)

                #fig.show()
                
                ylabel1 = ylabel + "per_m3_treated"
                
                fig = px.bar(sys_df, x="CaseStudy_Scenario", y="ByTreatedCapacity", color="Treatment Category",
                              labels={"Variable": "",  "ByTreatedCapacity": ylabel1},
                             color_discrete_sequence=px.colors.qualitative.T10,
                            template="simple_white")
                fig.update_layout(font=dict(size=18))
                
                fig.write_image("results/case_studies/%s/%s/figures/%s.png" % 
                                (source_water,file_type,(metric+"per_m3_treated")), 
                                format="png", engine="kaleido", width=900, height=700, scale=1.7)

                
                fig.show()
            

        for case_study in df["Case Study"].unique():
            
            if case_study is np.nan: continue
            
            sys_df = df[df["Case Study"] == case_study]

            for metric in ["Unit Levelized Cost", "Total Capital Investment (TCI)", "Annual O&M Costs", 
                          "Electricity Intensity System Treated"]:

                if metric == "Unit Levelized Cost": ylabel = "Levelized Cost of Water ($/m3)"
                if metric == "Total Capital Investment (TCI)": ylabel = "Total Capital Investment ($MM)"
                if metric == "Annual O&M Costs": ylabel = "Annual O&M Costs ($MM/yr)"
                if metric == "Electricity Intensity System Treated": ylabel = "Electricity Intensity kwh/m3 of Treated Water"

                sys_df1 = sys_df[sys_df["Unit Process Name"] != "System"]
                sys_df1 = sys_df1[sys_df1.Variable == metric]
                sys_df1 = sys_df1.groupby(["CaseStudy_Scenario", "Unit Process Name"]).sum()
                sys_df1 = sys_df1.reset_index()
                sys_df1 = sys_df1.sort_values("CaseStudy_Scenario", ascending=True)

                fig = px.bar(sys_df1, x="CaseStudy_Scenario", y="Value", color="Unit Process Name",
                              labels={"Variable": "",  "Value": ylabel},
                             color_discrete_sequence=px.colors.qualitative.Dark24,
                            template="simple_white")
                fig.update_layout(font=dict(size=18))
                
                for data in fig.data:
                    data["width"] = 0.5 #Change this value for bar widths

                fig.add_hline(y=0)
                
                fig.write_image("results/case_studies/%s/%s/by_unit/by_unit_%s_%s_%s.png" % (source_water,
                                                   file_type, case_study,metric, 
                                file_type),format="png", engine="kaleido", width=900, height=700, scale=1.7)
                
                sys_df.to_csv("results/case_studies/%s/%s/by_unit/by_unit_%s_%s_%s.csv" % (source_water,
                                      file_type, case_study,metric, file_type))

                #fig.show()
                
                if metric in ["Total Capital Investment (TCI)", "Annual O&M Costs"]:
                
                    ylabel1 = ylabel + "per_m3_treated"

                    fig = px.bar(sys_df, x="CaseStudy_Scenario", y="ByTreatedCapacity", color="Unit Process Name",
                                  labels={"Variable": "",  "ByTreatedCapacity": ylabel1},
                                 color_discrete_sequence=px.colors.qualitative.T10,
                                template="simple_white")      
                    fig.update_layout(font=dict(size=18))
                    
                    fig.write_image("results/case_studies/%s/%s/by_unit/by_unit_%s_%s.png" % 
                                    (source_water,file_type, case_study, (metric+"per_m3_treated")), 
                                    format="png", engine="kaleido", width=900, height=700, scale=1.7)
                    
                    
    #                 sys_df.to_csv("results/case_studies/%s/%s/csvs/%s_%s.csv" % (source_water, file_type, ylabel1, 
    #                                                                              file_type))




        ### SUMMARY TABLE FOR THE SYSTEM ####   
        sys_df = df[df["Unit Process Name"] == "System"]

        metrics = ["System Total Capital Investment (TCI)",
        "System Catalysts and Chemicals",
        "System Electricity",
        "System Total Operating Cost",
        "System Levelized Cost",
        "Electricity Intensity",
        "Water Recovery"]

        sys_df = sys_df[sys_df.Variable.isin(metrics)]
        del sys_df["Unit Process Name"]; del sys_df["Treatment Category"]; 
        del sys_df["Metric"]; del sys_df["case_name"];
        sys_df.to_csv("results/case_studies/%s/%s/summary_table_%s_%s.csv" % (source_water, 
                                                                file_type, source_water, file_type), index=False)
    
    

brackish
emwd emwd_zld_b.csv
emwd emwd_zld_c.csv
emwd emwd_zld_a.csv
emwd emwd_dwi.csv
kbhdp kbhdp_zld_a.csv
kbhdp kbhdp_zld_c.csv
kbhdp kbhdp_zld_b.csv
brackish ['emwd' 'irwin' 'kbhdp']
  Unit Process Name                              Variable     Value  \
0        well_field      Electricity Intensity Unit Inlet  0.000000   
1        well_field  Electricity Intensity System Treated  0.000000   
2        well_field                   Unit Levelized Cost  0.035550   
3        well_field        Fixed Capital Investment (FCI)  3.630801   
4        well_field                                  Land  0.005446   

        Metric    Unit Case Study  Scenario  
0  Electricity  kwh/m3       emwd  baseline  
1  Electricity  kwh/m3       emwd  baseline  
2         LCOW    $/m3       emwd  baseline  
3         Cost     $MM       emwd  baseline  
4         Cost     $MM       emwd  baseline  
adding inlet/outlet values EMWD
adding inlet/outlet values Irwin
adding inlet/outlet values KBHDP


  Unit Process Name                              Variable     Value  \
0        well_field      Electricity Intensity Unit Inlet  0.000000   
1        well_field  Electricity Intensity System Treated  0.000000   
2        well_field                   Unit Levelized Cost  0.035550   
3        well_field        Fixed Capital Investment (FCI)  3.630801   
4        well_field                                  Land  0.005446   

        Metric    Unit Case Study  Scenario  
0  Electricity  kwh/m3       emwd  baseline  
1  Electricity  kwh/m3       emwd  baseline  
2         LCOW    $/m3       emwd  baseline  
3         Cost     $MM       emwd  baseline  
4         Cost     $MM       emwd  baseline  
adding inlet/outlet values EMWD
adding inlet/outlet values EMWD zld_b
adding inlet/outlet values EMWD zld_c
adding inlet/outlet values EMWD zld_a
adding inlet/outlet values EMWD dwi
adding inlet/outlet values Irwin
adding inlet/outlet values KBHDP
adding inlet/outlet values KBHDP zld_a
adding inl

In [15]:
#### SENS

In [25]:
all_sens_df = pd.DataFrame()

df2 = pd.read_csv("data/baseline_cases_runs.csv")

for source_water in df2.source_water_category.unique():

    df = pd.read_csv("data/baseline_cases_runs.csv")
    cases = df[df.source_water_category == source_water].case_study.unique()
    source_sens_df = pd.DataFrame()
    for case in cases:
    
        df1 = pd.read_csv("results/case_studies/%s_baseline_sensitivity.csv" % case)

        df1["source_water"] = np.array(source_water)
        df1["Case Study"] = np.array(case)
        
        source_sens_df = pd.concat([source_sens_df,df1])
        all_sens_df = pd.concat([all_sens_df,df1])
    
    source_sens_df.to_csv("results/case_studies/%s/sensitivities/%s_sensitivities.csv" % (source_water, source_water))
    
del all_sens_df["Unnamed: 0"]

all_sens_df.to_csv("results/case_studies/all_sens_df.csv")


In [33]:
all_sens_df.head()

,lcow,water_recovery,elec_lcow,elec_int,scenario_value,scenario_name,lcow_difference,water_recovery_difference,elec_lcow_difference,source_water,Case Study
0,0.188578,96.0197,12.531263,0.360782,NaN,baseline,0.000000,0.0,0.000000e+00,industrial,iron_and_steel
1,0.269397,96.0197,12.531263,0.360782,0.700,Plant Capacity Utilization 70-100%,0.080819,0.0,-2.775558e-17,industrial,iron_and_steel
2,0.263746,96.0197,12.531263,0.360782,0.715,Plant Capacity Utilization 70-100%,0.075168,0.0,0.000000e+00,industrial,iron_and_steel
3,0.258326,96.0197,12.531263,0.360782,0.730,Plant Capacity Utilization 70-100%,0.069748,0.0,2.775558e-17,industrial,iron_and_steel
4,0.253125,96.0197,12.531263,0.360782,0.745,Plant Capacity Utilization 70-100%,0.064547,0.0,0.000000e+00,industrial,iron_and_steel


In [14]:
area_list = ["gila_river_both_ro_sensitivity.csv", 
"area_list_gila_river_both_ro_evaporation_pond.csv",
"gila_river_both_brine_sensitivity.csv",
"area_list_gila_river_both_brine_evaporation_pond.csv",
"gila_river_baseline_sensitivity.csv",
"area_list_gila_river_baseline_evaporation_pond.csv",
"cherokee_zld_brine_conc_sensitivity.csv",
"area_list_cherokee_zld_brine_conc_evaporation_pond.csv",
"cherokee_zld_ct_sensitivity.csv",
"area_list_cherokee_zld_ct_evaporation_pond.csv"]

In [32]:
df_final = pd.DataFrame()
for i in range(0, len(area_list), 2):
    print(i)
    df = pd.read_csv("results/case_studies/%s" % area_list[i])
    df1 = pd.read_csv("results/case_studies/%s" % area_list[i + 1])
    df_final[area_list[i]] = list(df[df.scenario_name == 'Air Temp 10-35 C'].lcow)
    df_final[area_list[i+1]] = list(df1["0"])

0
2
4
6
8


In [33]:
df_final.head()

,gila_river_both_ro_sensitivity.csv,area_list_gila_river_both_ro_evaporation_pond.csv,gila_river_both_brine_sensitivity.csv,area_list_gila_river_both_brine_evaporation_pond.csv,gila_river_baseline_sensitivity.csv,area_list_gila_river_baseline_evaporation_pond.csv,cherokee_zld_brine_conc_sensitivity.csv,area_list_cherokee_zld_brine_conc_evaporation_pond.csv,cherokee_zld_ct_sensitivity.csv,area_list_cherokee_zld_ct_evaporation_pond.csv
0,1.506148,207.577351,3.241900,64.822621,2.517259,133.358327,2.699810,29.204712,0.829547,46.386503
1,1.479939,199.957037,3.235497,62.442936,2.502676,128.462647,2.694602,28.132586,0.821033,44.683621
2,1.455568,192.876405,3.229539,60.231784,2.489113,123.913686,2.689754,27.136390,0.813111,43.101340
3,1.432847,186.280085,3.223982,58.171873,2.476466,119.675872,2.685229,26.208333,0.805718,41.627285
4,1.411613,180.120029,3.218785,56.248200,2.464644,115.718337,2.680995,25.341655,0.798805,40.250722


In [34]:
df_final.to_csv("results/evap_area.csv")

In [21]:
df.scenario_name.unique()

array(['zld_ct', 'Plant Capacity Utilization 70-100%',
       'Weighted Average Cost of Capital 5-10%', 'Inlet TDS +-25%',
       'Inlet Flow +-20%', 'Plant Lifetime 15-45 yrs',
       'electricity price +- 30%', 'Air Temp 10-35 C'], dtype=object)

In [26]:
df = pd.read_csv("results/case_studies/%s" % area_list[i+1])

In [ ]:
# # Create subplots: use 'domain' type for Pie subplot
# fig = make_subplots(rows=1, cols=len(sys_df1.CaseStudy_Scenario.unique()), specs=[size_dict])
# i = 1
# for case_scenario in sys_df1.CaseStudy_Scenario.unique():
#     dis_df = sys_df1[sys_df1.CaseStudy_Scenario == case_scenario]
    
#     fig = go.Figure(data=[go.Pie(labels=dis_df["Unit Process Name"], values=dis_df.Value, hole=.3)])
#     fig.update_layout(title_text=case_scenario)
        
#     fig.show()

# #     dis_df = sys_df1[sys_df1.CaseStudy_Scenario == case_scenario]
# #     fig.add_trace(go.Pie(labels=dis_df["Unit Process Name"], values=dis_df.Value, name=case_scenario),1, i)
# #     i = i +1
    
# # Use `hole` to create a donut-like pie chart
# #fig.update_traces(hole=.4, hoverinfo="label+percent+name")

# # fig.update_layout(
# #     title_text="Global Emissions 1990-2011",
# #     # Add annotations in the center of the donut pies.
# #     annotations=[dict(text='GHG', x=0.18, y=0.5, font_size=20, showarrow=False),
# #                  dict(text='CO2', x=0.82, y=0.5, font_size=20, showarrow=False)])
# #fig.show()